In [10]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import io
import cv2
import textwrap
import matplotlib.patches as patches
import math
import os
from pathlib import Path
from datetime import datetime
import json
import glob

seed = 42
np.random.seed(seed)

In [11]:
img_dir_path = Path("output", "images")
lbl_dir_path = Path("output", "labels")
today = f'{(int(datetime.now().strftime("%m%d")) - int("1001")):04}'
imgfn_prefix = f"image_{today}_"
lblfn_prefix = f"label_{today}"
startno = 0
endno = 1e8
wiki_maxnum = 100


In [12]:
def print_img(img, cvf = False):
    if cvf:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        # img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    plt.figure(figsize=(10, 10))#figsize=(5,5))
    plt.imshow(img)

In [ ]:
def checkImg(file_path):
    pass

def trans(pts1):
    res = []
    w, h = 640, 480
    for p in pts1:
        newp = [p[0], h-p[1]]
        res.append(newp)
    return np.float32(res)

def get_filenames(img_dir_path):
    res = glob.glob(str(img_dir_path / f"{imgfn_prefix}*.png"))
    return res

def read_json(json_file_path):  
    with open(json_file_path, 'r') as f:  
        data = json.load(f)  
    return data 

def calculate_distance(point1, point2):  
    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2) 

def main():
    filenames = get_filenames(img_dir_path)
    jsondata = read_json(str(lbl_dir_path / f"{lblfn_prefix}.json"))
    width, height = 640, 480
    json_pt = 0
    for i in range(10):
        checkFlag = (i >= startno and i < endno)
        if not checkFlag:
            continue
        # print_img(img)
        # fbn = str(filenames[i])
        #cv2.imshow(f'Input {fbn}', img)
        if not jsondata[json_pt]["image_name"] in filenames[i]:
            continue
        
        img =  cv2.imread(filenames[i])
        img = cv2.resize(img, (width, height))
        print_img(img, True)

        for sl in jsondata[json_pt]["string_labels"]:
            # id = sl["id"]
            pts1 = np.float32(sl["points"])
            dx = calculate_distance(pts1[0], pts1[2])
            dy = calculate_distance(pts1[0], pts1[1])
            p = dx/dy
            pts1 = trans(pts1)
            pts2 = np.float32([[width, height],[width,0],[0,0],[0, height]])
            matrix = cv2.getPerspectiveTransform(pts1, pts2)

            result = cv2.warpPerspective(img, matrix, (width, height))
            if dx >= dy:
                result = cv2.resize(result, (int(height*p), height))
            else:
                result = cv2.resize(result, (width, int(width/p)))
            print_img(result, True)
        json_pt += 1

    # cv2.destroyAllWindows() 

main()
